In [1]:
import os, numpy as np, nibabel as nib, matplotlib.pylab as plt

In [2]:
master_path = r'./BraTS/BraTS2021_Training_Data'
folders = [folder for folder in os.listdir(master_path) if folder != '.DS_Store']

In [33]:
if not os.path.exists(os.path.join(master_path + '_array')):
    os.makedirs(os.path.join(master_path + '_array'))

for counter, folder in enumerate(folders):
    path_img = os.path.join(master_path, folder, folder + '_flair.nii.gz')
    path_label = os.path.join(master_path, folder, folder + '_seg.nii.gz')
    img = nib.load(path_img).get_fdata()
    label = nib.load(path_label).get_fdata()

    if not os.path.exists(os.path.join(master_path + '_array', folder)):
        os.makedirs(os.path.join(master_path + '_array', folder))
    # break images down
    for i in range(img.shape[-1]):
        img_slice = img[:,:,i]
        label_slice = label[:,:,i]

        # Separate labels
        label_slice1 = np.where(label_slice == 1, 1, 0)
        label_slice2 = np.where(label_slice == 2, 1, 0)
        label_slice4 = np.where(label_slice == 4, 1, 0)

        # Save images using cv2
        np.save(os.path.join(master_path + '_array', folder, folder + '_flair' + '_' + str(i+1)), img_slice)

        plt.imsave(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1) + '_level_1.png'), label_slice1, cmap="gray")
        plt.imsave(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1) + '_level_2.png'), label_slice2, cmap="gray")
        plt.imsave(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1) + '_level_4.png'), label_slice4, cmap="gray")
        # np.save(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1)), label_slice)
    print(f'Finished {counter+1}/{len(folders)}: {folder}')


Finished 1/1251: BraTS2021_00639
Finished 2/1251: BraTS2021_01149
Finished 3/1251: BraTS2021_00606
Finished 4/1251: BraTS2021_01182
Finished 5/1251: BraTS2021_00262
Finished 6/1251: BraTS2021_01176
Finished 7/1251: BraTS2021_01344
Finished 8/1251: BraTS2021_00296
Finished 9/1251: BraTS2021_01512
Finished 10/1251: BraTS2021_00068
Finished 11/1251: BraTS2021_01388
Finished 12/1251: BraTS2021_01515
Finished 13/1251: BraTS2021_01343
Finished 14/1251: BraTS2021_00291
Finished 15/1251: BraTS2021_01171
Finished 16/1251: BraTS2021_01185
Finished 17/1251: BraTS2021_00433
Finished 18/1251: BraTS2021_00601
Finished 19/1251: BraTS2021_01178
Finished 20/1251: BraTS2021_00298
Finished 21/1251: BraTS2021_00608
Finished 22/1251: BraTS2021_01375
Finished 23/1251: BraTS2021_00095
Finished 24/1251: BraTS2021_01147
Finished 25/1251: BraTS2021_01523
Finished 26/1251: BraTS2021_00405
Finished 27/1251: BraTS2021_00253
Finished 28/1251: BraTS2021_01381
Finished 29/1251: BraTS2021_00061
Finished 30/1251: BraTS

In [ ]:
# To load an image
# np.load('/tmp/123.npy')